# Multiscale Analysis of the Sustainable Production of 1,3PDO and 3HP
## X1 - Demonstration of None Existence of Alternative Optimum at Economy Scale

In [49]:
%reset -f
%pylab inline

import cPickle as pickle
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Flux Variability Analysis of the Economy Scale Model

#### Non-Existence of Alternative Optima
- In a metabolic model, there are often many alternative optima
    - this is because there are many loops and equivalent alternative pathways in the model
- In the economy scale network model
    - there are very few loops
    - there are no equivalent alternative pathways
        - each pathway has a set of distinct yield, titer, productivity, and energy cost
    - as a result, **there are no alternative optima**

#### To demonstrate this, we performed the following "flux variability analysis"
- select Scenario 1A as the test scenario
    - minimization of global warming potential
    - 1000 kg PDO production, natrual gas heavy energy blend, 1 hectare land use
- calculate the optimal objective function
- bound the objective function flux to its optimal value
- maximize and minimize every other flux in the model
- we show that there is no variability in the fluxes when the objective flux is fixed at its optimum

- FVA using other scenarios (not included here) produce the same no variability result

In [50]:
import cPickle as pickle

from framed.analysis.simulation import FBA, pFBA
from framed.analysis.variability import FVA, flux_envelope_3d
from framed.hack.ratio_constraint import *
import matplotlib.pyplot as plt

from collections import OrderedDict

import matplotlib.pyplot as plt

import re

from mpl_toolkits.axes_grid1 import host_subplot

In [51]:
model = pickle.load(open('models/mini_BChI_model.pickle','rb'))

In [ ]:
## Setting up scenario 1A

### Objective Functions ###
OBJs=OrderedDict()
MAX = True
MIN = False
OBJs['min_global_warming'] = ('EX_co2e_mkt', MIN) # minimize total global warming potential

### Flux Constraints ###
FCs = OrderedDict()
FCs['no import'] = {'DM_glyc_mkt':(0,0), 'DM_glc_mkt':(0,0), 'DM_energy_mkt':(0,0)}
FCs['product demand: 3hp = 1000']= {'DM_3hp_mkt':(1000,1000), 'DM_1,3pdo_mkt':(0,0)}
FCs['product demand: pdo = 1000']= {'DM_3hp_mkt':(0, 0), 'DM_1,3pdo_mkt':(1000,1000)}
FCs['product demand: 3hp = 500, pdo=500']= {'DM_3hp_mkt':(500,500), 'DM_1,3pdo_mkt':(500,500)}

FCs['positive profits']= {'PROFIT_res':(0,None), 'PROFIT_agr':(0,None), 'PROFIT_erg':(0,None), 'PROFIT_bch':(0,None)}
FCs['no biodiesel'] = {'energy_from_biodiesel':(0,0)}
FCs['land use < 1'] = {'EX_land_res':(-1,0)}
FCs['land use < 5'] = {'EX_land_res':(-5,0)}
FCs['co2 < 2000'] = {'EX_co2e_mkt':(0, 2000)}


### Ratio Constraints ###

# EU Energy Mix
def add_RC_EU_energy(model):
    add_ratio_constraint(model, 'energy_from_petroleum', 'MX_erg2mkt_energy', 0.25)
    add_ratio_constraint(model, 'energy_from_natural_gas', 'MX_erg2mkt_energy', 0.7)
    add_ratio_constraint(model, 'energy_from_coal', 'MX_erg2mkt_energy', 0.05)

def remove_RC_EU_energy(model):
    remove_ratio_constraint(model, 'ratio_energy_from_petroleum_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_natural_gas_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_coal_MX_erg2mkt_energy')

Scenarios = OrderedDict()
base_scenario = {k:v for d in (FCs['no import'], FCs['positive profits']) for k, v in d.iteritems()}
Scenarios['pdo1000, land<1, energy=EU'] = {k:v for d in (base_scenario, FCs['product demand: pdo = 1000'], FCs['land use < 1'], FCs['no biodiesel']) for k, v in d.iteritems()}


In [ ]:
# Simulating Scenario 1A Flux Variability
import copy

O = 'min_global_warming'
S = 'pdo1000, land<1, energy=EU'

if 'energy=EU' in S:
    add_RC_EU_energy(model)  # add EU energy mixing constraint

solutions = FBA(model, objective={OBJs[O][0]:1}, maximize=OBJs[O][1], constraints = Scenarios[S])

# fix objective flux to its optimal value
cons = copy.copy(Scenarios[S])
fobj = abs(solutions.fobj)
cons[OBJs[O][0]] = (fobj ,fobj)
reactions = model.reactions.keys()
variability = OrderedDict([(r_id, [None, None]) for r_id in reactions])

for rxn in reactions:
#     minimize flux rxn
    vfa_sol0 = FBA(model, {rxn: -1}, True, constraints=cons)
    variability[rxn][0] = -vfa_sol0.fobj
    
#     maximize flux rxn
    vfa_sol1 = FBA(model, {rxn: 1}, True, constraints=cons)
    variability[rxn][1] = vfa_sol1.fobj


### FVA shows that there is no variability in the fluxes when the objective function flux is fixed

In [ ]:
# print flux if the maximum value is different from the minimum value
    # set 10^-11 as the boundary for numerical error
for r in variability.keys():
    if abs(variability[r][0] - variability[r][1]) > 10**-11:
        print r, ': ', variability[r]

In [ ]:
# print all fluxes
for r in variability.keys():
    print r, ': ', variability[r]